In [ ]:
# !pip install evaluate seqeval datasets sacremoses matplotlib transformers
# !pip install torch==2.0.0
# !pip install --upgrade accelerate

In [2]:
from TokenClassificationTrainer import TokenClassificationTrainer
import pandas as pd
import numpy as np
import torch

In [ ]:
def runs(discriminate_lr = False, scheduler = False, save_name = "baseline"):
    # Set the task and name of the pretrained model and the batch size for finetuning
    task = "ner"
    model_name = "xlm-mlm-17-1280"  # "bert-base-multilingual-cased" or "xlm-mlm-17-1280"
    seed = np.random.randint(0, 1000)
    save_name = save_name + ".seed-" + str(seed)
    batch_size = 32

    # Flag to indicate whether to label all tokens or just the first token of each word
    label_all_tokens = True

    # File paths to splits of the chosen dataset
    file_paths = {
        "train": "data/datasets/ewt/en_ewt_nn_train.conll",
        "validation": "data/datasets/ewt/en_ewt_nn_newsgroup_dev.conll",
    }

    # initialize trainer
    trainer = TokenClassificationTrainer(task, model_name, save_name, batch_size, label_all_tokens, file_paths)


    # Training
    trainer.train(discriminate_lr = discriminate_lr, scheduler = scheduler, seed = seed)

    evals = trainer.evaluate_multiple(["data/datasets/ewt/en_ewt_nn_newsgroup_test.conll", "data/datasets/NoSta-D/NER-de-test.tsv", "data/datasets/DaNplus/da_news_test.tsv", "data/datasets/hungarian/hungarian_test.tsv"])

    baseline_eval_baseline_model = evals[0]
    NoStaD_eval_baseline_model = evals[1]
    DaNplus_eval_baseline_model = evals[2]
    Hungarian_eval_baseline_model = evals[3]

    cols = ["Dataset", "Language", "Seed"] + [name for name, _ in baseline_eval_baseline_model.items()]

    df = pd.DataFrame(columns=cols)

    # Add the evals to df
    df.loc[0] = ["Baseline", "English", seed] + [value for _, value in baseline_eval_baseline_model.items()]
    df.loc[1] = ["NoSta-D", "German", seed] + [value for _, value in NoStaD_eval_baseline_model.items()]
    df.loc[2] = ["DaNplus", "Danish", seed] + [value for _, value in DaNplus_eval_baseline_model.items()]
    df.loc[3] = ["Hungarian", "Hungarian", seed] + [value for _, value in Hungarian_eval_baseline_model.items()]

    torch.cuda.empty_cache()

    return df


In [7]:
list_baseline = []
list_discriminate_lr = []
list_scheduler = []
list_both = []
for i in range(10):
    list_baseline.append(runs(save_name = "baseline"))
    list_discriminate_lr.append(runs(discriminate_lr = True, save_name = "discriminate_lr"))
    list_scheduler.append(runs(scheduler = True, save_name = "scheduler"))
    list_both.append(runs(discriminate_lr = True, scheduler = True, save_name = "both"))
